# Readme

This is a file for the final exam in the course called "Python for Genomic Data Science". The Focus of this file is on the analysis of ORFs and repeats. This file answers all questions 100% correctly.

## Import data

In [1]:
from Bio import SeqIO
from Bio.Seq import Seq

In [2]:
my_seq = list(SeqIO.parse("dna2.fasta", "fasta"))

In [3]:
my_seq[0]

SeqRecord(seq=Seq('CTCGCGTTGCAGGCCGGCGTGTCGCGCAACGACGTGTGGGGCCTGACGGGCAGG...GCC'), id='gi|142022655|gb|EQ086233.1|91', name='gi|142022655|gb|EQ086233.1|91', description='gi|142022655|gb|EQ086233.1|91 marine metagenome JCVI_SCAF_1096627390048 genomic scaffold, whole genome shotgun sequence', dbxrefs=[])

## How many my_seq are in the multi-FASTA file?

In [4]:
len(my_seq)

18

## What is the length of the longest sequence in the file?

In [5]:
maximum = 0
for i in range(len(my_seq)):
    record_len = len(my_seq[i].seq)
    print(record_len)
    if record_len >= maximum:
        maximum = record_len

print(f"This is the longest string {maximum}")

4635
1151
4894
3511
4076
2867
442
890
967
4338
1352
4564
4804
964
2095
1432
115
2646
This is the longest string 4894


## What is the length of the shortest sequence in the file?

In [6]:
minimum = maximum
for i in range(len(my_seq)):
    record_len = len(my_seq[i].seq)
    print(record_len)
    if record_len <= minimum:
        minimum = record_len

print(f"This is the shortest string {minimum}")

4635
1151
4894
3511
4076
2867
442
890
967
4338
1352
4564
4804
964
2095
1432
115
2646
This is the shortest string 115


## What is the length of the longest ORF appearing in reading frame 2 of any of the sequences?

An open reading frame (ORF) is the part of a reading frame that has the potential to encode a protein. It starts with a start codon (ATG), and ends with a stop codon (TAA, TAG or TGA). For instance, ATGAAATAG is an ORF of length 9.

An open reading frame (ORF) is a sequence of DNA or RNA that does not contain any stop codons (TAA, TAG, or TGA) and can potentially encode a protein.

In [7]:
# checks whether a sequence has a stop codon
# assumes that the reading frame is 1, which means that
# the reading starts with the very first nucleotide
"""
input - DNA sequence
output - True (detected) or False (not detected)
"""

def check_orf(seq):
    
    # stop codons
    stop_codons = ["TAA", "TAG", "TGA"]
    
    N = len(seq)
    
    # raise Exception("ERROR, because not divisible by 3")
    if N % 3 != 0:        
        return "ERROR, because not divisible by 3"
    
    # MAIN FUNCTION
    # find stop codons in the middle of a sequence
    for i in range(0, N-6, 3):
        
        if seq[i:i+3] in stop_codons:
            
            return(f"NOT ORF index {i} and {seq[i:i+3]}")
    
    return "This is ORF"

In [8]:
# divides into subsequences starting with ATG and ending with TAA, TAG or TGA
# they are called ORF subsequences
# it is not allowed to have the stop codons in the middle
# subsequences must be longer than 6 nucleotides
"""
input: a DNA sequence, reading frame (1,2,3)
output: ORF subsequences
"""

def split_orf_seq(seq, k=0):
    
    # stop codons
    stop_codons = ["TAA", "TAG", "TGA"]
    
    sub_seq = []
    ind_seq = []
    
    N = len(seq)
    
    # frame must be 1,2 or 3 which means that k must be 0,1 or 2
    if k not in [0,1,2]:
        raise ValueError("k must be equal to 0, 1 or 2")

    # MAIN FUNCTION
    for i in range(k, N-k-3, 3):
        
        if seq[i:i+3] == 'ATG':
            
            # IMPORTANT!!!
            # step must be 3 in range
            for j in range(i+3, N, 3):
                if seq[j:j+3] in stop_codons:
                    
                    sub_seq_ = seq[i:j+3]
                    # sub_seq_ must be longer than 6 nucleotides
                    if len(sub_seq_) > 6:
                        sub_seq.append(sub_seq_)
                        ind_seq.append((i,j+3, j-i+3))
                    
                    #IMPORTANT!!!
                    # when stop codon is detected, we need to break from the cycle
                    break
                    

    return sub_seq, ind_seq

In [9]:
# check two functions above on a small example
dna_seq = Seq("AATGAAAATGAAATAAATGAAATGA")
sequences, indices = split_orf_seq(dna_seq,1)
for seq, indices in zip(sequences, indices):
    print(seq)
    print(indices)
    print(check_orf(seq))

ATGAAAATGAAATAA
(1, 16, 15)
This is ORF
ATGAAATAA
(7, 16, 9)
This is ORF
ATGAAATGA
(16, 25, 9)
This is ORF


In [10]:
all_indices = []

for i in range(len(my_seq)):
    
    seq = my_seq[i].seq
    
    sequences, indices = split_orf_seq(seq,1)
    
    print(f"For {my_seq[i].id}, there are ORFs {indices}")
    
    all_indices.append(indices)

For gi|142022655|gb|EQ086233.1|91, there are ORFs [(76, 127, 51), (292, 343, 51), (346, 355, 9), (640, 655, 15), (658, 748, 90), (817, 1054, 237), (1102, 1135, 33), (1414, 1555, 141), (1657, 1666, 9), (1747, 1819, 72), (2077, 2215, 138), (2395, 2416, 21), (2557, 2584, 27), (2860, 2881, 21)]
For gi|142022655|gb|EQ086233.1|304, there are ORFs []
For gi|142022655|gb|EQ086233.1|255, there are ORFs [(70, 157, 87), (88, 157, 69), (109, 157, 48), (205, 256, 51), (211, 256, 45), (286, 295, 9), (430, 1615, 1185), (565, 1615, 1050), (589, 1615, 1026), (745, 1615, 870), (862, 1615, 753), (880, 1615, 735), (1387, 1615, 228), (1855, 1867, 12), (2032, 2209, 177), (2035, 2209, 174), (2059, 2209, 150), (2119, 2209, 90), (2377, 2512, 135), (2395, 2512, 117), (2425, 2512, 87), (2464, 2512, 48), (2467, 2512, 45), (2485, 2512, 27), (2527, 3553, 1026), (2572, 3553, 981), (2626, 3553, 927), (2650, 3553, 903), (2779, 3553, 774), (2956, 3553, 597), (2995, 3553, 558), (3121, 3553, 432), (3139, 3553, 414), (321

In [11]:
all_indices[0][0][2]

51

In [12]:
maximum = 0
value = ()

for i in range(len(all_indices)):
    
    for j in range(len(all_indices[i])):
        
        if all_indices[i][j][2] > maximum:
            maximum = all_indices[i][j][2]
            value = all_indices[i][j]

print(maximum)
print(value)

1458
(3070, 4528, 1458)


## What is the starting position of the longest ORF in reading frame 3 in any of the sequences? The position should indicate the character number where the ORF begins.

In [13]:
all_indices = []

for i in range(len(my_seq)):
    
    seq = my_seq[i].seq
    
    sequences, indices = split_orf_seq(seq,2)
    
    print(f"For {my_seq[i].id}, there are ORFs {indices}")
    
    all_indices.append(indices)

For gi|142022655|gb|EQ086233.1|91, there are ORFs [(908, 1262, 354), (1400, 1439, 39), (2270, 2399, 129), (2855, 3443, 588), (3089, 3443, 354), (3386, 3443, 57), (3449, 3533, 84), (3542, 3692, 150), (3563, 3692, 129), (3650, 3692, 42), (3752, 3851, 99), (3797, 3851, 54), (3941, 3956, 15), (4082, 4130, 48), (4262, 4292, 30), (4310, 4376, 66), (4391, 4472, 81)]
For gi|142022655|gb|EQ086233.1|304, there are ORFs [(620, 767, 147), (623, 767, 144), (908, 1028, 120), (1097, 1139, 42)]
For gi|142022655|gb|EQ086233.1|255, there are ORFs [(776, 947, 171), (1178, 1187, 9), (1640, 1925, 285), (1691, 1925, 234), (1754, 1925, 171), (1817, 1925, 108), (2021, 2033, 12), (2609, 2675, 66), (3704, 3905, 201), (3803, 3905, 102), (4025, 4217, 192)]
For gi|142022655|gb|EQ086233.1|45, there are ORFs [(215, 473, 258), (362, 473, 111), (431, 473, 42), (524, 557, 33), (698, 1022, 324), (2627, 2711, 84)]
For gi|142022655|gb|EQ086233.1|396, there are ORFs [(428, 653, 225), (434, 653, 219), (1940, 1970, 30), (199

In [14]:
maximum = 0
value = ()

for i in range(len(all_indices)):
    
    for j in range(len(all_indices[i])):
        
        if all_indices[i][j][2] > maximum:
            maximum = all_indices[i][j][2]
            value = all_indices[i][j]

print(maximum)
print(value)

1821
(635, 2456, 1821)


## What is the length of the longest ORF appearing in any sequence and in any forward reading frame?

In [15]:
all_indices = []

for i in range(len(my_seq)):
    
    seq = my_seq[i].seq
    
    sequences, indices = split_orf_seq(seq,0)
    
    print(f"For {my_seq[i].id}, there are ORFs {indices}")
    
    all_indices.append(indices)

For gi|142022655|gb|EQ086233.1|91, there are ORFs [(228, 906, 678), (453, 906, 453), (588, 906, 318), (978, 2274, 1296), (1071, 2274, 1203), (1083, 2274, 1191), (1194, 2274, 1080), (1200, 2274, 1074), (2055, 2274, 219), (2226, 2274, 48), (2385, 3093, 708), (2454, 3093, 639), (2583, 3093, 510), (3744, 4032, 288)]
For gi|142022655|gb|EQ086233.1|304, there are ORFs [(519, 588, 69), (858, 963, 105)]
For gi|142022655|gb|EQ086233.1|255, there are ORFs [(291, 1734, 1443), (297, 1734, 1437), (345, 1734, 1389), (768, 1734, 966), (1290, 1734, 444), (1395, 1734, 339), (1725, 1734, 9), (1863, 2493, 630), (1866, 2493, 627), (2502, 2568, 66), (2580, 2613, 33)]
For gi|142022655|gb|EQ086233.1|45, there are ORFs [(384, 2778, 2394), (681, 2778, 2097), (705, 2778, 2073), (711, 2778, 2067), (879, 2778, 1899), (975, 2778, 1803), (1002, 2778, 1776), (1122, 2778, 1656), (2757, 2778, 21)]
For gi|142022655|gb|EQ086233.1|396, there are ORFs [(144, 408, 264), (204, 408, 204), (354, 408, 54), (528, 1587, 1059), (

In [16]:
maximum = 0
value = ()

for i in range(len(all_indices)):
    
    for j in range(len(all_indices[i])):
        
        if all_indices[i][j][2] > maximum:
            maximum = all_indices[i][j][2]
            value = all_indices[i][j]

print(maximum)
print(value)

2394
(384, 2778, 2394)


## What is the length of the longest forward ORF that appears in the sequence with the identifier  gi|142022655|gb|EQ086233.1|16?

It is 1644. I did it with Ctrl + F.

## Find the most frequently occurring repeat of length 6 in all sequences. How many times does it occur in all?

In [17]:
# Finds how many repeats of a given length within a single sequence
# Repeats can overlap
"""
input - sequence (seq), length (l)
output - dict[seq] = amount of repeats
"""

def repeats_one_seq(seq, l=6):
    
    repeats = {}
    
    N = len(seq)
    
    for i in range(N-l+1):
        
        rep_seq = seq[i:i+l]
               
        if rep_seq not in repeats.keys():

            repeats[rep_seq] = 0
            
            for j in range(N):

                if rep_seq == seq[j:j+l]:
                    repeats[rep_seq] += 1
    
    return repeats



In [18]:
repeats_one_seq('AAABBB', l=2)

{'AA': 2, 'AB': 1, 'BB': 2}

In [19]:
# this script merges two dictionaries together
# by adding values
"""
input - main_dict and merge_dict
output - main_dict will be updated in-place
"""

def merge_dict_seq(main_dict, merge_dict):
    
    # check all keys
    for key in merge_dict.keys():
        if key in main_dict.keys():
            main_dict[key] = main_dict[key] + merge_dict[key]
        else:
            main_dict[key] = merge_dict[key]
    
    

In [20]:
list_all_seq = []

for i in range(len(my_seq)):
    list_all_seq.append(my_seq[i].seq)
    
list_all_seq

[Seq('CTCGCGTTGCAGGCCGGCGTGTCGCGCAACGACGTGTGGGGCCTGACGGGCAGG...GCC'),
 Seq('CGCGATCCAGTAGCGCTTGTAGCCGAGCGCTTCGGCACGCTTCGCGAGCGCGAT...GCC'),
 Seq('CTCGACGCGCTCCGCGTCGAGGTCGCCCGACGTCTCGCGCAGCAACTGATTCAA...TCG'),
 Seq('CGTGCTCGGCACGACTATCAGCCCGTATCTGTTTTTCTGGCAGGCCTCCCAGGA...CCG'),
 Seq('TGCATCGGCGACGGCTACTGGTTCCTCGAGAAAGCGCGCAACTACGCGAGCGAG...TCA'),
 Seq('GCCCGTGCCGACGCGCCGCATCGTCAGCGCGCGGCGCTCGTGGTCCTGCCACGC...CGA'),
 Seq('TGCGGGCGGCCTGTGGGCGGTCGGCGCCGGGCTGTCGCAGCCGCTGTTCCACGG...GCG'),
 Seq('GGACCGGCGAGGAAGTGGTCGGCGTCGAACAGGACGATCGCTGCGTCACGTTGC...CTC'),
 Seq('GCCGCGGGCGCCGGACGGAGCACGCGCATGATCCCGCTTTCCCATCCGTCCTTC...TCG'),
 Seq('ACTGCCGCGTTCAGCGCGAGAATGTTCGTCTGGAACGCAATCCCTTCGATTACC...GAA'),
 Seq('TATAGGCCTGGACCTGCGTGCGGGCGAGATCGGCGTCTTCGCCGAAGGTGCCGT...GAG'),
 Seq('TCGCGGGCCTCGCGCTGCCGTTCGCGGCGCTCGCGTTGCTGGAGGTGGTCGTGC...CAG'),
 Seq('GTCGATCGACACGACGCTCGCGCAGCGCGACGCGAAGGCCGCGTGAGCGCACGA...CGT'),
 Seq('CGCAGGTGGCAGCATTTAAGATCTCGCAAGGTGGTGTCTCCCCCATCATTGGTG...GGC'),
 Seq('ATGTGCCTGAAGAC

In [21]:
repeats = {}

for a_seq in list_all_seq:
    
    dict_seq = repeats_one_seq(a_seq, l=6)
    
    merge_dict_seq(repeats, dict_seq)
    


In [22]:
maximum = 0

for key in repeats.keys():
    
    if repeats[key] > maximum:
        maximum = repeats[key]
        max_key = key

In [23]:
print(max_key, repeats[max_key])

GCGCGC 153


## Find all repeats of length 12 in the input file. 
Let's use Max to specify the number of copies of the most frequent repeat of length 12. How many different 12-base sequences occur Max times?

In [24]:
repeats = {}

for a_seq in list_all_seq:
    
    dict_seq = repeats_one_seq(a_seq, l=12)
    
    merge_dict_seq(repeats, dict_seq)
    

In [32]:
maximum = 0

for key in repeats.keys():
    
    if repeats[key] > maximum:
        maximum = repeats[key]
        max_key = key

In [33]:
print(max_key, repeats[max_key])

CATTCGCCATTC 10


In [34]:
for key in repeats.keys():
    
    if repeats[key] == 10:
        print(key)

CATTCGCCATTC
ATTCGCCATTCG
TTCGCCATTCGC
TCGCCATTCGCC


## Which one of the following repeats of length 7 has a maximum number of occurrences?

In [35]:
repeats = {}

for a_seq in list_all_seq:
    
    dict_seq = repeats_one_seq(a_seq, l=7)
    
    merge_dict_seq(repeats, dict_seq)

In [36]:
maximum = 0

for key in repeats.keys():
    
    if repeats[key] > maximum:
        maximum = repeats[key]
        max_key = key

In [37]:
print(max_key, repeats[max_key])

CGCGCCG 63
